In [1]:
import pandas as pd

df = pd.read_csv('data/train_time.csv')
df = df[:100]
df.head()

,id,sentence,date,birthday
0,100022_0,Болеет СД 2 типа в течении 5 лет.,2010-12-14 18:33:23,1957-01-01
1,100085_0,"Считает себя больной с 1993 года, когда перене...",2010-12-14 21:06:48,1950-11-05
2,100085_4,В мае 2009 года в СибГМУ проводилась РЧ-аблаци...,2010-12-14 21:06:48,1950-11-05
3,100085_5,"Ухудшение в конце 2009, начале 2010 года, когд...",2010-12-14 21:06:48,1950-11-05
4,100085_7,В апреле 2010 года проходила лечение в отд НРС...,2010-12-14 21:06:48,1950-11-05


In [6]:
from syntax.parser import Parser

parser = Parser()


[nltk_data] Downloading package punkt to /home/renoip2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
Initialization complete!


# Parse single sentence

In [7]:
from TimeExpressions.TimeProcessor import TimeProcessor

processor = TimeProcessor()

In [4]:
doc = processor.process(sentence="Болеет СД 2 типа в течении 5 лет .", parser=parser)
print([('ВК: '+ent.text, 'Нормальная форма: '+str(ent._.normal_form), 'Событие: '+ent._.event) for ent in doc[0].ents])

[('ВК: в течении 5 лет', 'Нормальная форма: [datetime.datetime(2015, 12, 25, 8, 34, 50, 224118), datetime.datetime(2020, 12, 25, 8, 34, 50, 224118)]', 'Событие: Болеет СД 2 типа')]


# Parse set of sentences and convert result to pandas

In [1]:
import TimeExpressions.TimeProcessor

In [2]:
import imp

imp.reload(TimeExpressions.TimeProcessor)
processor = TimeExpressions.TimeProcessor.TimeProcessor()

In [3]:
import pandas as pd

df = pd.read_csv('data/weather_data.csv')
df = df[520:]
df.tail()

,phrase,value
1148,утром следующего дня он скончался,утром следующего дня
1149,еще накануне вечером она собиралась жить,накануне вечером
1150,а в продаже диски появятся уже завтра,завтра
1151,утром стали известны итоги выборов,утром
1152,утром проблемные места почистить пемзой,утром


In [4]:
with open("2020-12-25 10:39:14.conll", "r") as f:
    text = f.read()

parsed_sentences = text.split("\n\n")

In [5]:
result = processor.process(parsed_sentences=parsed_sentences, to_dataframe=True)
result = result[520:]
df['result'] = result.time_expr
df = df.fillna('')
from sklearn.metrics import accuracy_score
pd.set_option('display.max_rows', 1000)

print(accuracy_score(df.value, df.result))
print(df[df.value!=df.result].shape[0])
df[df.value!=df.result]

0.5892575039494471
260


,phrase,value,result
520,"хотелось бы узнать , что за погода будет завтр...",завтра и послезавтра,"завтра, послезавтра"
524,"меня интересует , какая погода будет ближайшие...",ближайшие два дня,два дня
525,"мне хочется знать , в ближайшие дни какая пого...",в ближайшие дни,
526,"а какая погодка будет в ближайшую неделю , ска...",в ближайшую неделю,
527,а что у нас с погодой вечером,вечером,
528,погода какая ожидается вечером после 6,вечером после 6,
529,какая будет погода в районе 8 часов вечера,в районе 8 часов вечера,8 часов вечера
530,сегодня и в следующие 2 дня какая будет погода,сегодня и в следующие 2 дня,"сегодня, 2 дня"
533,какая погода ожидается на выходных,на выходных,
534,какая погода будет в предстоящие праздники,в предстоящие праздники,


In [12]:
result = processor.process(sentence=df.sentence.values, date=df.date.values, birthday=df.birthday.values, to_dataframe=True, parser=parser)
result.head(10)

AttributeError: 'DataFrame' object has no attribute 'sentence'

In [6]:
result.uncertain[2]

'[2009-04-15 00:00:00, 2009-05-15 00:00:00, 2009-06-15 00:00:00]'

# Example of negation module

In [7]:
import pandas as pd

df_neg = pd.read_csv('data/train_neg.csv', index_col=0)
df_neg = df_neg[:100]
df_neg.head()

,sentences,form,entity
0,Со слов больного ранее боли в области сердца н...,"боли не отмечал, АД не контролировал","боли, АД"
1,"На ЭКГ элевация ST V1-V3 , с представлением об...",не принимает медикаменты,медикаменты
2,За время наблюдения самочувствие без ухудшения...,"без ухудшения самочувствие, без особенностей м...",ухудшения
3,"Не обследовалась , антиангинальной терапии не ...","Не обследовалась, не проводилось антиангинальн...",обследовалась
4,На контрольной КГ- отмечается некоторое усугуб...,не выполнять дилатацию устья,дилатацию устья


In [8]:
from negations import negations

negator = negations.Negator()

In [9]:
result = negator.process(sentence=df_neg.sentences.values, parser=parser, to_dataframe=True)
result.head(10)

,sentence,neg_expr,neg_ent
0,Со слов больного ранее боли в области сердца н...,"не отмечал, не контролировал АД","None, АД"
1,"На ЭКГ элевация ST V1-V3 , с представлением об...",не принимает медикаменты,медикаменты
2,За время наблюдения самочувствие без ухудшения...,"без ухудшения самочувствие, без особенностей м...","ухудшения самочувствие, особенностей место пун..."
3,"Не обследовалась , антиангинальной терапии не ...","Не обследовалась, не проводилось антиангинальн...","None, антиангинальной терапии"
4,На контрольной КГ - отмечается некоторое усугу...,не выполнять дилатацию устья,дилатацию устья
5,На фоне терапии кордароном ЖНР не рецидивирова...,На фоне терапии кордароном ЖНР не рецидивирова...,"На фоне терапии кордароном ЖНР, None, ангиноз..."
6,По данным СМ - ЭКГ - за время наблюдения регис...,не зарегистрированы нарушения,нарушения
7,"С 2010 г - ГБ ( АДmax - 195/100mmHg ) , без ре...",без терапии,терапии
8,"В ФЦСКЭ не обращался , антиангинальную терапию...","В ФЦСКЭ не обращался, не получал антиангинальн...","В ФЦСКЭ, антиангинальную терапию"
9,"лозап 50 мг , статины ) сположительным эффекто...",не рецидивировали боли,боли
